# Sample MaterialX Notebook (JSON test)

In [1]:
import MaterialX as mx
import mtlxutils.mxfile as mxf
import json

from IPython.display import display_markdown

# Print xmltodict version
print('Materialx version: ', mx.__version__)
print('Json version: ', json.__version__)


Materialx version:  1.38.8
Json version:  2.0.9


In [31]:
def getShaderNodes(graphElement):
    '''
    Find all surface shaders in a GraphElement.
    '''
    shaderNodes = set()
    for material in graphElement.getMaterialNodes():
        for shader in mx.getShaderNodes(material):
            shaderNodes.add(shader.getNamePath())
    for shader in graphElement.getNodes():
        if shader.getType() == 'surfaceshader':
            shaderNodes.add(shader.getNamePath())
    return shaderNodes


def getRenderableGraphs(graphElement):
    '''
    Find all renderable graphs in a GraphElement.
    '''
    ngnamepaths = set()
    graphs = []
    shaderNodes = getShaderNodes(graphElement)
    for shaderPath in shaderNodes:
        shader = doc.getDescendant(shaderPath)
        for input in shader.getInputs():
            ngString = input.getNodeGraphString()
            if ngString and ngString not in ngnamepaths:
                graphs.append(graphElement.getNodeGraph(ngString))
                ngnamepaths.add(ngString)
    return graphs

In [32]:
def copyGraphInterfaces(dest, ng):
    '''
    Copy the interface of a nodegraph to a new nodegraph under a specified parent `dest`.
    '''
    copyMethod = 'add_remove'
    ng1 = dest.addNodeGraph(ng.getName())
    if copyMethod == 'add_remove':
        ng1.copyContentFrom(ng)
        for child in ng1.getChildren():
            if child.isA(mx.Input) or child.isA(mx.Output):
                for attr in ['nodegraph', 'nodename', 'defaultinput']:
                    child.removeAttribute(attr)
                continue
            ng1.removeChild(child.getName())
    else:
        for attrName in ng.getAttributeNames():
            attr = ng.getAttribute(attrName)
            newattr = ng1.addAttribute(attr.getName(), attr.getType(), attr.getValue())
            newattr.copyContentFrom(attr)
        for port in ng.getInputs():
            newport = ng1.addInput(port.getName(), port.getType())
            newport.copyContentFrom(port)
        for port in ng.getOutputs():
            newport = ng1.addOutput(port.getName(), port.getType())
            newport.copyContentFrom(port)
            for attr in ['nodegraph', 'nodename', 'defaultinput']:
                newport.removeAttribute(attr)    

In [33]:
# We use a colon to separate the category and name of an element in the JSON hierarchy
JSON_CATEGORY_NAME_SEPARATOR = ':'
# The root of the JSON hierarchy
MATERIALX_DOCUMENT_ROOT = 'materialx'

# Convert MaterialX element to JSON
def elementToJSON(elem, jsonParent):
    '''
    Convert an MaterialX XML element to JSON.
    Will recursively traverse the parent/child Element hierarchy.
    '''
    if (elem.getSourceUri() != ""):
        return
    
    # Create a new JSON element for the MaterialX element
    jsonElem = {}

    # Add attributes
    for attrName in elem.getAttributeNames():
        jsonElem[attrName] = elem.getAttribute(attrName)

    # Add children
    for child in elem.getChildren():
        jsonElem = elementToJSON(child, jsonElem)
    
    # Add element to parent
    jsonParent[elem.getCategory() + JSON_CATEGORY_NAME_SEPARATOR + elem.getName()] = jsonElem
    return jsonParent

# Convert MaterialX document to JSON
def documentToJSON(doc):
    '''Convert an MaterialX XML document to JSON'''
    root = {}
    root["materialx"] = {}

    for attrName in doc.getAttributeNames():
        root[attrName] =  doc.getAttribute(attrName)

    for elem in doc.getChildren():
        elementToJSON(elem, root[MATERIALX_DOCUMENT_ROOT])

    result = json.dumps(root, indent=2)
    return result

In [34]:
# Load in sample MaterialX file
#doc = mx.createDocument()
doc, libFiles, status = mxf.MtlxFile.createWorkingDocument()
mx.readFromXmlFile(doc, './data/sample_marble.mtlx')

# Create destination document and copy nodegraph interfaces over
xmldoc = mx.createDocument()
graphs = getRenderableGraphs(doc)
for ng in graphs:
    copyGraphInterfaces(xmldoc, ng)

# Get interfaces as an XML string
xml_string = mxf.MtlxFile.writeDocumentToString(xmldoc)

text = '<details><summary>Extracted MaterialX Nodegraphs in XML</summary>\n\n' + '```xml\n' + xml_string + '\n```\n' + '</details>\n' 
display_markdown(text, raw=True)

filename = './data/sample_marble_graph.mtlx'
mx.writeToXmlFile(xmldoc, filename)

<details><summary>Extracted MaterialX Nodegraphs in XML</summary>

```xml
<?xml version="1.0"?>
<materialx version="1.38">
  <nodegraph name="NG_marble1">
    <input name="base_color_1" type="color3" value="0.8, 0.8, 0.8" uiname="Color 1" uifolder="Marble Color" />
    <input name="base_color_2" type="color3" value="0.1, 0.1, 0.3" uiname="Color 2" uifolder="Marble Color" />
    <input name="noise_scale_1" type="float" value="6.0" uisoftmin="1.0" uisoftmax="10.0" uiname="Scale 1" uifolder="Marble Noise" />
    <input name="noise_scale_2" type="float" value="4.0" uisoftmin="1.0" uisoftmax="10.0" uiname="Scale 2" uifolder="Marble Noise" />
    <input name="noise_power" type="float" value="3.0" uisoftmin="1.0" uisoftmax="10.0" uiname="Power" uifolder="Marble Noise" />
    <input name="noise_octaves" type="integer" value="3" uisoftmin="1" uisoftmax="8" uiname="Octaves" uifolder="Marble Noise" />
    <output name="out" type="color3" />
  </nodegraph>
</materialx>

```
</details>


In [35]:
# Convert entire document
doc_result = documentToJSON(doc)

text = '<details><summary>Entire document to JSON</summary>\n\n' + '```json\n' + doc_result  + '\n```\n' + '</details>\n' 
display_markdown(text, raw=True)

# Convert just the graph
graph_result = documentToJSON(xmldoc)

text = '<details><summary>Node Graph Interface to JSON</summary>\n\n' + '```json\n' + graph_result  + '\n```\n' + '</details>\n' 
display_markdown(text, raw=True)

<details><summary>Entire document to JSON</summary>

```json
{
  "materialx": {
    "nodegraph:NG_marble1": {
      "input:base_color_1": {
        "type": "color3",
        "value": "0.8, 0.8, 0.8",
        "uiname": "Color 1",
        "uifolder": "Marble Color"
      },
      "input:base_color_2": {
        "type": "color3",
        "value": "0.1, 0.1, 0.3",
        "uiname": "Color 2",
        "uifolder": "Marble Color"
      },
      "input:noise_scale_1": {
        "type": "float",
        "value": "6.0",
        "uisoftmin": "1.0",
        "uisoftmax": "10.0",
        "uiname": "Scale 1",
        "uifolder": "Marble Noise"
      },
      "input:noise_scale_2": {
        "type": "float",
        "value": "4.0",
        "uisoftmin": "1.0",
        "uisoftmax": "10.0",
        "uiname": "Scale 2",
        "uifolder": "Marble Noise"
      },
      "input:noise_power": {
        "type": "float",
        "value": "3.0",
        "uisoftmin": "1.0",
        "uisoftmax": "10.0",
        "uiname": "Power",
        "uifolder": "Marble Noise"
      },
      "input:noise_octaves": {
        "type": "integer",
        "value": "3",
        "uisoftmin": "1",
        "uisoftmax": "8",
        "uiname": "Octaves",
        "uifolder": "Marble Noise"
      },
      "position:obj_pos": {
        "type": "vector3"
      },
      "dotproduct:add_xyz": {
        "type": "float",
        "input:in1": {
          "type": "vector3",
          "nodename": "obj_pos"
        },
        "input:in2": {
          "type": "vector3",
          "value": "1, 1, 1"
        }
      },
      "multiply:scale_xyz": {
        "type": "float",
        "input:in1": {
          "type": "float",
          "nodename": "add_xyz"
        },
        "input:in2": {
          "type": "float",
          "interfacename": "noise_scale_1"
        }
      },
      "multiply:scale_pos": {
        "type": "vector3",
        "input:in1": {
          "type": "vector3",
          "nodename": "obj_pos"
        },
        "input:in2": {
          "type": "float",
          "interfacename": "noise_scale_2"
        }
      },
      "fractal3d:noise": {
        "type": "float",
        "input:octaves": {
          "type": "integer",
          "interfacename": "noise_octaves"
        },
        "input:position": {
          "type": "vector3",
          "nodename": "scale_pos"
        }
      },
      "multiply:scale_noise": {
        "type": "float",
        "input:in1": {
          "type": "float",
          "nodename": "noise"
        },
        "input:in2": {
          "type": "float",
          "value": "3.0"
        }
      },
      "add:sum": {
        "type": "float",
        "input:in1": {
          "type": "float",
          "nodename": "scale_xyz"
        },
        "input:in2": {
          "type": "float",
          "nodename": "scale_noise"
        }
      },
      "sin:sin": {
        "type": "float",
        "input:in": {
          "type": "float",
          "nodename": "sum"
        }
      },
      "multiply:scale": {
        "type": "float",
        "input:in1": {
          "type": "float",
          "nodename": "sin"
        },
        "input:in2": {
          "type": "float",
          "value": "0.5"
        }
      },
      "add:bias": {
        "type": "float",
        "input:in1": {
          "type": "float",
          "nodename": "scale"
        },
        "input:in2": {
          "type": "float",
          "value": "0.5"
        }
      },
      "power:power": {
        "type": "float",
        "input:in1": {
          "type": "float",
          "nodename": "bias"
        },
        "input:in2": {
          "type": "float",
          "interfacename": "noise_power"
        }
      },
      "mix:color_mix": {
        "type": "color3",
        "input:bg": {
          "type": "color3",
          "interfacename": "base_color_1"
        },
        "input:fg": {
          "type": "color3",
          "interfacename": "base_color_2"
        },
        "input:mix": {
          "type": "float",
          "nodename": "power"
        }
      },
      "output:out": {
        "type": "color3",
        "nodename": "color_mix"
      }
    },
    "standard_surface:SR_marble1": {
      "type": "surfaceshader",
      "xpos": "13.768116",
      "ypos": "-0.672414",
      "input:base": {
        "type": "float",
        "value": "1"
      },
      "input:base_color": {
        "type": "color3",
        "nodegraph": "NG_marble1",
        "output": "out"
      },
      "input:specular_roughness": {
        "type": "float",
        "value": "0.1"
      },
      "input:subsurface": {
        "type": "float",
        "value": "0.4"
      },
      "input:subsurface_color": {
        "type": "color3",
        "nodegraph": "NG_marble1",
        "output": "out"
      }
    },
    "surfacematerial:Marble_3D": {
      "type": "material",
      "xpos": "17.391304",
      "ypos": "0.000000",
      "input:surfaceshader": {
        "type": "surfaceshader",
        "nodename": "SR_marble1"
      }
    }
  },
  "version": "1.38",
  "colorspace": "lin_rec709"
}
```
</details>


<details><summary>Node Graph Interface to JSON</summary>

```json
{
  "materialx": {
    "nodegraph:NG_marble1": {
      "input:base_color_1": {
        "type": "color3",
        "value": "0.8, 0.8, 0.8",
        "uiname": "Color 1",
        "uifolder": "Marble Color"
      },
      "input:base_color_2": {
        "type": "color3",
        "value": "0.1, 0.1, 0.3",
        "uiname": "Color 2",
        "uifolder": "Marble Color"
      },
      "input:noise_scale_1": {
        "type": "float",
        "value": "6.0",
        "uisoftmin": "1.0",
        "uisoftmax": "10.0",
        "uiname": "Scale 1",
        "uifolder": "Marble Noise"
      },
      "input:noise_scale_2": {
        "type": "float",
        "value": "4.0",
        "uisoftmin": "1.0",
        "uisoftmax": "10.0",
        "uiname": "Scale 2",
        "uifolder": "Marble Noise"
      },
      "input:noise_power": {
        "type": "float",
        "value": "3.0",
        "uisoftmin": "1.0",
        "uisoftmax": "10.0",
        "uiname": "Power",
        "uifolder": "Marble Noise"
      },
      "input:noise_octaves": {
        "type": "integer",
        "value": "3",
        "uisoftmin": "1",
        "uisoftmax": "8",
        "uiname": "Octaves",
        "uifolder": "Marble Noise"
      },
      "output:out": {
        "type": "color3"
      }
    }
  },
  "version": "1.38"
}
```
</details>


In [36]:
# Save graph_result to file
filename = './data/sample_marble_graph.json'
file = open(filename, 'w')
file.write(graph_result)
file.close()

# Save doc_result to file
filename = './data/sample_marble.json'
file = open(filename, 'w')
file.write(doc_result)
file.close()


##### Deserialization from JSON

For conversion from JSON we introduce two functions:

1. `documentFromJSON()` : Converts a JSON string to a MaterialX document.
2. `elementFromJSON()` : Converts a JSON string to a MaterialX element, and continues to recursively convert any children.

Note that to perform deserialization we need to split the `category` and `name` out for non-attribute elements.

In [37]:
# Separator between category and name in JSON element
JSON_CATEGORY_NAME_SEPARATOR = ":"

# Convert JSON element to MaterialX
def elementFromJSON(node, elem):
    '''
    Convert an JSON element to MaterialX
    '''
    for key in node:
        value = node[key]

        # Set attributes            
        if isinstance(value, str):
            elem.setAttribute(key, str(value))

        # Traverse chilren
        else:
            # Traverse down from root
            if key == MATERIALX_DOCUMENT_ROOT:
                elementFromJSON(value, elem)
                continue

            # Split key name by ":" to get category and name
            category, name = key.split(JSON_CATEGORY_NAME_SEPARATOR, 1)
            if category and not elem.getChild(name):
                child = elem.addChildOfCategory(category, name)
                elementFromJSON(value, child)

# Convert JSON to MaterialX document
def documentFromJSON(jsonDoc, doc):
    '''
    Convert a JSON document to MaterialX
    '''
    elementFromJSON(jsonDoc, doc)

In [38]:
# Convert entire document back from JSON
newDoc = mx.createDocument() 
jsonObject = json.loads(doc_result)
documentFromJSON(jsonObject, newDoc)

# Validate and upgrade element version
valid, errors = newDoc.validate()
if not valid:
    print('Validation errors:')
    for err in errors:
        print('  {}'.format(err))
newDoc.upgradeVersion()

newDocString = mx.writeToXmlString(newDoc)    
text = '<details><summary>JSON Deserialization of Document</summary>\n\n' + '```xml\n' + newDocString  + '\n```\n' + '</details>\n' 
display_markdown(text, raw=True)

<details><summary>JSON Deserialization of Document</summary>

```xml
<?xml version="1.0"?>
<materialx version="1.38" colorspace="lin_rec709">
  <nodegraph name="NG_marble1">
    <input name="base_color_1" type="color3" value="0.8, 0.8, 0.8" uiname="Color 1" uifolder="Marble Color" />
    <input name="base_color_2" type="color3" value="0.1, 0.1, 0.3" uiname="Color 2" uifolder="Marble Color" />
    <input name="noise_scale_1" type="float" value="6.0" uisoftmin="1.0" uisoftmax="10.0" uiname="Scale 1" uifolder="Marble Noise" />
    <input name="noise_scale_2" type="float" value="4.0" uisoftmin="1.0" uisoftmax="10.0" uiname="Scale 2" uifolder="Marble Noise" />
    <input name="noise_power" type="float" value="3.0" uisoftmin="1.0" uisoftmax="10.0" uiname="Power" uifolder="Marble Noise" />
    <input name="noise_octaves" type="integer" value="3" uisoftmin="1" uisoftmax="8" uiname="Octaves" uifolder="Marble Noise" />
    <position name="obj_pos" type="vector3" />
    <dotproduct name="add_xyz" type="float">
      <input name="in1" type="vector3" nodename="obj_pos" />
      <input name="in2" type="vector3" value="1, 1, 1" />
    </dotproduct>
    <multiply name="scale_xyz" type="float">
      <input name="in1" type="float" nodename="add_xyz" />
      <input name="in2" type="float" interfacename="noise_scale_1" />
    </multiply>
    <multiply name="scale_pos" type="vector3">
      <input name="in1" type="vector3" nodename="obj_pos" />
      <input name="in2" type="float" interfacename="noise_scale_2" />
    </multiply>
    <fractal3d name="noise" type="float">
      <input name="octaves" type="integer" interfacename="noise_octaves" />
      <input name="position" type="vector3" nodename="scale_pos" />
    </fractal3d>
    <multiply name="scale_noise" type="float">
      <input name="in1" type="float" nodename="noise" />
      <input name="in2" type="float" value="3.0" />
    </multiply>
    <add name="sum" type="float">
      <input name="in1" type="float" nodename="scale_xyz" />
      <input name="in2" type="float" nodename="scale_noise" />
    </add>
    <sin name="sin" type="float">
      <input name="in" type="float" nodename="sum" />
    </sin>
    <multiply name="scale" type="float">
      <input name="in1" type="float" nodename="sin" />
      <input name="in2" type="float" value="0.5" />
    </multiply>
    <add name="bias" type="float">
      <input name="in1" type="float" nodename="scale" />
      <input name="in2" type="float" value="0.5" />
    </add>
    <power name="power" type="float">
      <input name="in1" type="float" nodename="bias" />
      <input name="in2" type="float" interfacename="noise_power" />
    </power>
    <mix name="color_mix" type="color3">
      <input name="bg" type="color3" interfacename="base_color_1" />
      <input name="fg" type="color3" interfacename="base_color_2" />
      <input name="mix" type="float" nodename="power" />
    </mix>
    <output name="out" type="color3" nodename="color_mix" />
  </nodegraph>
  <standard_surface name="SR_marble1" type="surfaceshader" xpos="13.768116" ypos="-0.672414">
    <input name="base" type="float" value="1" />
    <input name="base_color" type="color3" nodegraph="NG_marble1" output="out" />
    <input name="specular_roughness" type="float" value="0.1" />
    <input name="subsurface" type="float" value="0.4" />
    <input name="subsurface_color" type="color3" nodegraph="NG_marble1" output="out" />
  </standard_surface>
  <surfacematerial name="Marble_3D" type="material" xpos="17.391304" ypos="0.000000">
    <input name="surfaceshader" type="surfaceshader" nodename="SR_marble1" />
  </surfacematerial>
</materialx>

```
</details>
